In [ ]:
## TODO ensure this script performs properly if the number of participants is not a
## multiple of 5 - we should then just create more organizations/nodes without user
# accounts to let them have the same experience as the others
url = "https://server.workshop.vantage6.ai"
port = 443
api_path = "/api"

# When running this, create a file config.py with the following content:
from config import username, password

In [ ]:
from vantage6.client import Client

client = Client(url, port, api_path, log_level='info')

client.authenticate(username, password)
client.setup_encryption(None)

### Register algorithm store

In [ ]:
# TODO: I now commente this out because only needs to be done once - make it resilient
#client.store.create(algorithm_store_url="https://store.cotopaxi.vantage6.ai", name="Community store",
#                    all_collaborations=True)

### Define user details

In [ ]:
# TODO replace participants with actual data
from workshop_resources.organizations import organizations
from workshop_resources.collaborations import collaborations as collaboration_names
from workshop_resources.collaborations import studies as study_names
from workshop_resources.collaborations import agot_studies
from workshop_resources.collaborations import gga_studies

In [ ]:
### read participants as list of dictionaries
import csv

participants_file = 'workshop_resources/participants_real.csv'
participants = []
with open(participants_file, mode='r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        participants.append(row)
print(len(participants))

In [ ]:
i = 0
while len(participants) % 6 != 0:
    participants.append({
        'first_name': f'John-{i}',
        'last_name': f'Doe-{i}',
        'email': f'john-doe-{i}@dummy.co'
    })
    i += 1

print(f"Creating {len(participants)} participants of which {i} dummies")

### Register organizations

In [ ]:
org_data = []
for org in organizations:
    org_data.append(client.organization.create(**org, address2=''))
    print(f"Created organization {org['name']}")

In [ ]:
import random
import string

def generate_password(length=8):
    characters = string.ascii_letters + string.digits
    password = ''.join(random.choices(characters, k=length))
    return password + '1aA!'

In [ ]:
import pandas as pd

# create users
researcher_role = 5
user_data = []
for i, participant in enumerate(participants):
    username = participant['first_name'][0].lower() + participant['last_name'].lower()
    print(f"--> Creating user {username}")
    password = generate_password(8)
    data = {
        "firstname": participant['first_name'],
        "lastname": participant['last_name'],
        "email": participant['email'],
        # username of John Doe is jdoe
        "username": username,
        "password": password,
        "roles": [researcher_role],
        "rules": [],
        "organization": org_data[i]['id']
    }
    client.user.create(**data)
    user_data.append(data)

df = pd.DataFrame(user_data)
df.to_csv('workshop_resources/credentials.csv', index=False)

In [ ]:
SIZE_COLLABS = 5

In [ ]:
# create collaborations
# Users are assigned to the first 35 organizations, then there are 7 at the end that
# are offline. We will group users in collaborations of 6 organizations, 5 from the
# first 35 and 1 from the last 7 (offline) organizations.
collaboration_data = []
collaboration_responses = []
for idx in range(0, int(len(participants) / SIZE_COLLABS)):
    data = {
        "name": collaboration_names[idx],
        "organizations": [
            org_data[j]['id'] for j in range(idx * SIZE_COLLABS, (idx*SIZE_COLLABS)+SIZE_COLLABS)
        ] + [
            org_data[len(participants) + idx]['id']
        ],
        "encrypted": False
    }
    collaboration_data.append(data)
    response = client.collaboration.create(**data)
    collaboration_responses.append(response)

In [ ]:
from pathlib import Path
from jinja2 import Environment, FileSystemLoader

# define function to create node config files from template
def create_node_config_file(api_key, node_name, nth_node_in_collab):
    environment = Environment(
        loader=FileSystemLoader("templates"),
        trim_blocks=True,
        lstrip_blocks=True,
        autoescape=True,
    )
    template = environment.get_template("node_config.j2")

    node_config = template.render(
        api_key=api_key,
        default_data_file_path=f"/data/default_dataset_{nth_node_in_collab}.csv",
        km_data_file_path=f"/data/km_dataset_{nth_node_in_collab}.csv",
        log_file=f"{node_name}.log",
        task_dir=f"/home/vantage6/.local/share/vantage6/node/{node_name}"
    )

    cwd = Path.cwd()
    output_path = Path(cwd / "output" / "node_configs" / f"{node_name}.yaml")
    output_path.parent.mkdir(parents=True, exist_ok=True)
    with open(output_path, "w") as f:
        f.write(node_config)


In [ ]:
# collaboration_responses = client.collaboration.list(scope="global")["data"]
# collaboration_data = []
# for collab in collaboration_responses:
#     collaboration_data.append({
#         "name": collab['name'],
#         "organizations": [org["id"] for org in client.organization.list(collaboration=collab['id'], per_page=999)["data"]],
#         "encrypted": False
#     })


In [ ]:
# org_data = client.organization.list(per_page=999)["data"]
# org_data

In [ ]:
# register nodes
node_responses = []
for idx in range(0, len(collaboration_data)):
    data = {
        "collaboration": collaboration_responses[idx]['id'],
    }
    for org_idx, org in enumerate(collaboration_data[idx]['organizations']):
        data['organization'] = org
        org_data_match = [org_data[i] for i in range(len(org_data)) if org_data[i]['id'] == org]
        data["name"] = (collaboration_data[idx]['name'].split(' ')[0] + "_" + org_data_match[0]['name']).split(' ')[0]
        response = client.node.create(**data)
        # for all nodes except the last one, create a node config file
        if org_idx != SIZE_COLLABS:
            create_node_config_file(response['api_key'], data["name"], org_idx + 1)
        node_responses.append(response)
        print(f"Created node {data['name']}")

In [ ]:
# create studies. These should contain all organizations in the collaboration except the
# last one, which is offline.
for collab_name in collaboration_names:
    # TODO we are not sure here that all the collaboration_names are created as we are
    # not sure that we have sufficient participants. However it will raise an error,
    # this is not a problem as the studies for the necessary collaborations are created
    collab = client.collaboration.list(scope="global", name=collab_name, per_page=999)['data'][0]
    orgs = client.organization.list(collaboration=collab['id'], per_page=999)['data']
    org_ids = [org['id'] for org in orgs]
    # remove the org with the highest id, which is the offline org

    # subset, only the online nodes
    subset = [id_ for id_ in org_ids if id_ != max(org_ids)]

    # 4 online nodes and 1 offline
    gga_subset = subset[1:] + [max(org_ids)]

    # 5 online nodes
    agot_subset = subset.copy()

    def create_study(lookup_study_name_table, org_ids):
        data = {
            "name": lookup_study_name_table[collaboration_names.index(collab_name)],
            "collaboration": collab['id'],
            "organizations": org_ids,
        }
        client.study.create(**data)


    create_study(study_names, subset)
    create_study(agot_studies, agot_subset)
    create_study(gga_studies, gga_subset)


In [ ]:
# node_responses = []
# for idx in range(0, len(collaboration_data)):
#     data = {
#         "collaboration": collaboration_responses[idx]['id'],
#     }
#     for org in collaboration_data[idx]['organizations']:
#         data['organization'] = org
#         # TODO remove name in favor of auto-naming when client is fixed
#         data["name"] = f"Node {org}"
#         response = client.node.create(**data)
#         node_responses.append(response)

# Register dummy resources

The following should register the following resources:

- 1 user per participant with workshop admin role
- Each user should be in a new dummy organization, and there should be 2 extra organizations
- There should be 2 collaborations, both containing the user's organization and one of the extra organizations



In [ ]:
len(participants)

In [ ]:
# create organizations
org_data = {}
dummy_org = {'address1': '', 'address2': '', 'country': '', 'domain': '', 'zipcode': ''}
for participant in participants:
    abbrev_name = participant['first_name'][0].lower() + participant['last_name'].lower()
    partipant_orgs = [
        client.organization.create(
            name=f"{abbrev_name}_main_org",
            **dummy_org
        ),
        client.organization.create(
            name=f"{abbrev_name}_sub_org1",
            **dummy_org
        ),
        client.organization.create(
            name=f"{abbrev_name}_sub_org2",
            **dummy_org
        ),
    ]
    org_data[abbrev_name] = partipant_orgs

In [ ]:
# Read credentials from CSV file
credentials_file = 'workshop_resources/credentials.csv'
credentials = []
with open(credentials_file, mode='r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        credentials.append(row)

In [ ]:
for participant in participants:
    for credential in credentials:
        if participant['email'] == credential['email']:
            participant["password"] = credential["password"]

In [ ]:
# create users
workshopadmin_role = 8
for participant in participants:
    abbrev_name = participant['first_name'][0].lower() + participant['last_name'].lower()
    org_id = org_data[abbrev_name][0]['id']
    data = {
        "firstname": participant['first_name'],
        "lastname": participant['last_name'],
        "email": participant['email']+ "2",
        # username of John Doe is jdoe_admin
        "username": abbrev_name + "_admin",
        "password": participant['password'],
        "roles": [workshopadmin_role],
        "rules": [],
        "organization": org_id
    }
    client.user.create(**data)

In [ ]:
for participant in participants:
    abbrev_name = participant['first_name'][0].lower() + participant['last_name'].lower()
    participant_orgs = org_data[abbrev_name]
    # create one collaboration for first and second org
    data = {
        "name": f"{abbrev_name}_collab1",
        "organizations": [participant_orgs[0]['id'], participant_orgs[1]['id']],
        "encrypted": False
    }
    response = client.collaboration.create(**data)
    # create one collaboartion for first and third org
    data = {
        "name": f"{abbrev_name}_collab2",
        "organizations": [participant_orgs[0]['id'], participant_orgs[2]['id']],
        "encrypted": False
    }
    response = client.collaboration.create(**data)